In [ ]:
import copy
import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import importlib
import life.lib
import life.lib as lib
import typing
importlib.reload(lib)
LOG = lib.simple_log.LOG
import math
from matplotlib import pyplot as plt
%matplotlib
plt.ion()
lib.utils.set_seed()
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4



In [ ]:
lib.utils.set_seed()
downscale = (14, 14)
test_x, test_y = lib.datasets.to_plain(*lib.datasets.get_mnist_test(), downscale=downscale, to_odd=True)
train_x, train_y = lib.datasets.to_plain(*lib.datasets.get_mnist_train(), downscale=downscale, to_odd=True)
network = lib.livenet.LiveNet.create_perceptron(train_x.shape[1], None, 2)

In [ ]:
batch_iterator = lib.gen_utils.batch_iterator(train_x, train_y, batch_size=1000)
criterion = lib.nets.criterion_n
optimizer = lib.nets.create_optimizer(network)
optimizer.learning_rate = 0.01
trainer = lib.trainer.Trainer(network, batch_iterator, criterion, optimizer, epoch_size=100)


In [ ]:
network.context.regularization_l1 = 0.01
trainer.step(25000)


In [ ]:
pred = network(test_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = test_y - pred_bin
len(diff[diff != 0]) / len(diff)

In [ ]:
importlib.reload(lib)
%matplotlib
plt.ion()

def param_amount(param):
    val = len(param["params"])
    return val

def param_max(param):
    val = np.max(np.abs(param["params"]))
    return val

def param_picker1(param):
    try:
        val0 = param["params"]["S262->D1"].item()
    except KeyError:
        val0 = 0.
    return val0

def get_param_values(history, picker):
    values = []
    for entry in history:
        values.append(picker(entry))
    return values

plt.figure(figsize=(16, 9))
values = get_param_values(trainer.history, param_amount)
plt.plot(values)
plt.grid()

accum = lib.stat_utils.AccumStat()
accum.add_value(trainer.network.parameters())
# accum.plot()

In [ ]:
for name, p in sorted(network.named_parameters(),key=lambda x: len(x[0]) + ord(x[0][0]) / 1000.):
    print(name, f"{p.item():.1f}")

In [ ]:
network._parameters

In [ ]:
lib.livenet.export_onnx(network, "/home/spometun/model.onnx")